In [2]:
import sys
sys.path.append('../')
from mytapering import *

In [3]:
import pennylane as qml
from pennylane import qchem

import numpy as np
from time import time

In [4]:
np.set_printoptions(precision=6)

# 2. Example: LiH

In [5]:
# https://github.com/aoterodelaroza/refdata/blob/master/20_g3/lih.xyz
symbols = ["Li", "H"]
coordinates = np.array([0.0, 0.0, 0.403635, 0.0, 0.0, -1.210905])

In [6]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  12
The Hamiltonian is 
   (-3.844095589078443) [I0]
+ (-0.38874433443438816) [Z11]
+ (-0.38874433443438805) [Z10]
+ (-0.21337079370277998) [Z9]
+ (-0.21337079370277995) [Z8]
+ (-0.21337079370270737) [Z7]
+ (-0.21337079370270734) [Z6]
+ (-0.18269688137599668) [Z5]
+ (-0.18269688137599663) [Z4]
+ (-0.08811650933952661) [Z3]
+ (-0.0881165093395265) [Z2]
+ (1.0657693942968418) [Z0]
+ (1.0657693942968425) [Z1]
+ (-0.0040328077024712255) [Y1 Y3]
+ (-0.0040328077024712255) [X1 X3]
+ (0.001121895797431289) [Y3 Y5]
+ (0.001121895797431289) [X3 X5]
+ (0.008829856445608452) [Y2 Y4]
+ (0.008829856445608452) [X2 X4]
+ (0.04515772747839051) [Y0 Y2]
+ (0.04515772747839051) [X0 X2]
+ (0.05516443343995296) [Z4 Z10]
+ (0.05516443343995296) [Z5 Z11]
+ (0.05992853198294553) [Z4 Z6]
+ (0.05992853198294553) [Z5 Z7]
+ (0.059928531982973314) [Z4 Z8]
+ (0.059928531982973314) [Z5 Z9]
+ (0.06101038170335723) [Z2 Z4]
+ (0.06101038170335723) [Z3 Z5]
+ (0.06195795786087671) [Z4 Z11]
+ (0.0619579

## 2.1 Tapering using QML

In [7]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 4
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

C:\Users\Lee\anaconda3\envs\qiskit\lib\site-packages\pennylane\operation.py:1921: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
C:\Users\Lee\anaconda3\envs\qiskit\lib\site-packages\pennylane\operation.py:2111: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


time cost: 15.677810430526733
Number of qubit after tapering = 8
The tapered Hamiltonian is 
   ((-3.68762286203592+0j)) [I0]
+ ((-0.42674158740555956-1.0408340855860843e-17j)) [Z8]
+ ((-0.4267415874054145-1.1622647289044608e-16j)) [Z6]
+ ((-0.18269688137599657+1.734723475976807e-18j)) [Z4]
+ ((-0.18269688137599657+0j)) [Z5]
+ ((-0.08811650933952657+0j)) [Z3]
+ ((-0.08811650933952643+3.469446951953614e-18j)) [Z2]
+ ((-0.009731378415249032+0j)) [X5]
+ ((-0.0036224668189408647+0j)) [X0]
+ ((0.003329640392654342+0j)) [X6]
+ ((0.0033296403926558835+0j)) [X8]
+ ((0.009033626870508097+0j)) [X4]
+ ((0.030963141132992696+0j)) [X1]
+ ((1.0657693942968405+3.469446951953614e-18j)) [Z0]
+ ((1.0657693942968416+3.469446951953614e-18j)) [Z1]
+ ((-0.030731095558011737+0j)) [Y2 Y3]
+ ((-0.012871066449284539+0j)) [X4 Z5]
+ ((-0.0067935244209237486+0j)) [Y4 Y5]
+ ((-0.005052528617892372+0j)) [Y0 Y1]
+ ((-0.004032807702471221+0j)) [Y1 Y3]
+ ((-0.004032807702471221+0j)) [X1 X3]
+ ((-0.0026554679625305938+0

In [8]:
Hami_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered, 9)

-3.687622862036 IIIIIIIII
-0.388744334434 IZIZIZZIZ
-0.388744334434 ZIZIZIZIZ
 0.109141895865 ZZZZZZIII
-0.426741587406 IIIIIIIIZ
 0.134626888109 IZIZIZZII
 0.134626888109 ZIZIZIZII
-0.426741587405 IIIIIIZII
 0.134626888109 IZIZIZIIZ
 0.134626888109 ZIZIZIIIZ
 0.270772614652 IIIIIIZIZ
 1.065769394297 ZIIIIIIII
 0.106204389776 ZZIZIZZIZ
 0.101151861158 IIZIZIZIZ
 0.195473555618 ZIIIIIIIZ
 0.195473555618 ZIIIIIZII
 0.061010381703 IIZIZIIII
 1.065769394297 IZIIIIIII
 0.101151861158 IIIZIZZIZ
 0.106204389776 ZZZIZIZIZ
 0.195473555618 IZIIIIIIZ
 0.195473555618 IZIIIIZII
 0.061010381703 IIIZIZIII
 0.408201270075 ZZIIIIIII
 0.027247318533 YZYIIIIII
-0.004169326929 YIYZIZZIZ
 0.001513858966 XZXIZIZIZ
 0.005268688553 YZYIIIIIZ
 0.005268688553 YZYIIIZII
 0.004633602065 XZXIZIIII
 0.045157727478 YIYIIIIII
 0.027247318533 XZXIIIIII
-0.004169326929 XIXZIZZIZ
 0.001513858966 YZYIZIZIZ
 0.005268688553 XZXIIIIIZ
 0.005268688553 XZXIIIZII
 0.004633602065 YZYIZIIII
 0.045157727478 XIXIIIIII
 0.024240011

## 2.2 Tapering using mytarpering

In [9]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

-3.844095589078 IIIIIIIIIIII
 1.065769394297 ZIIIIIIIIIII
 1.065769394297 IZIIIIIIIIII
 0.408201270075 ZZIIIIIIIIII
 0.027247318533 YZYIIIIIIIII
 0.045157727478 YIYIIIIIIIII
 0.027247318533 XZXIIIIIIIII
 0.045157727478 XIXIIIIIIIII
 0.024240011239 YZZZYIIIIIII
 0.028997285065 YIZZYIIIIIII
 0.024240011239 XZZZXIIIIIII
 0.028997285065 XIZZXIIIIIII
-0.025836613617 YZZZZZZZZZYI
-0.030963141133 YIZZZZZZZZYI
-0.025836613617 XZZZZZZZZZXI
-0.030963141133 XIZZZZZZZZXI
 0.027247318533 IYZYIIIIIIII
 0.045157727478 ZYZYIIIIIIII
 0.027247318533 IXZXIIIIIIII
 0.045157727478 ZXZXIIIIIIII
 0.010434033303 YXXYIIIIIIII
-0.010434033303 YYXXIIIIIIII
-0.010434033303 XXYYIIIIIIII
 0.010434033303 XYYXIIIIIIII
-0.003299302880 YXIXYIIIIIII
-0.003299302880 YYIYYIIIIIII
-0.003299302880 XXIXXIIIIIII
-0.003299302880 XYIYXIIIIIII
 0.006617582989 YXIXZZZZZZYI
 0.006617582989 YYIYZZZZZZYI
 0.006617582989 XXIXZZZZZZXI
 0.006617582989 XYIYZZZZZZXI
 0.024240011239 IYZZZYIIIIII
 0.028997285065 ZYZZZYIIIIII
 0.02424001123

In [10]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, n_electrons)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

time cost: 2.1770999431610107
Number of qubit after tapering = 8
The tapered Hamiltonian is 
 -3.68762286+0.00000000j IIIIIIII
1.06576939+0.00000000j ZIIIIIII
1.06576939+0.00000000j IZIIIIII
0.40820127+0.00000000j ZZIIIIII
0.02724732+0.00000000j YZYIIIII
0.04515773+0.00000000j YIYIIIII
0.02724732+0.00000000j XZXIIIII
0.04515773+0.00000000j XIXIIIII
0.02424001+0.00000000j YZZZYIII
0.02899729+0.00000000j YIZZYIII
0.02424001+0.00000000j XZZZXIII
0.02899729+0.00000000j XIZZXIII
0.02583661+0.00000000j XZIZIZZZ
0.03096314+0.00000000j XIIZIZZZ
-0.02583661+0.00000000j XZZZZZII
-0.03096314+0.00000000j XIZZZZII
0.02724732+0.00000000j IYZYIIII
0.04515773+0.00000000j ZYZYIIII
0.02724732+0.00000000j IXZXIIII
0.04515773+0.00000000j ZXZXIIII
0.01043403+0.00000000j YXXYIIII
-0.01043403+0.00000000j YYXXIIII
-0.01043403+0.00000000j XXYYIIII
0.01043403+0.00000000j XYYXIIII
-0.00329930+0.00000000j YXIXYIII
-0.00329930+0.00000000j YYIYYIII
-0.00329930+0.00000000j XXIXXIII
-0.00329930+0.00000000j XYIYXIII
-

## 2.3 compare

### 2.3.1 generators and paulixops

In [11]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

generator 1:   (1.0) [Z6 Z7], paulix_op: PauliX(wires=[7])
generator 2:   (1.0) [Z8 Z9], paulix_op: PauliX(wires=[9])
generator 3:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10], paulix_op: PauliX(wires=[10])
generator 4:   (1.0) [Z1 Z3 Z5 Z6 Z8 Z11], paulix_op: PauliX(wires=[11])


In [12]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

generator 1: 1.00000000 IIIIIIZZIIII, paulix_op: 1.00000000 IIIIIIIXIIII
generator 2: 1.00000000 IIIIIIIIZZII, paulix_op: 1.00000000 IIIIIIIIIXII
generator 3: 1.00000000 ZIZIZIZIZIZI, paulix_op: 1.00000000 IIIIIIIIIIXI
generator 4: 1.00000000 IZIZIZZIZIIZ, paulix_op: 1.00000000 IIIIIIIIIIIX


### 2.3.2 optimal sectors

In [13]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')

sectors by qml: [1, 1, 1, 1]
sectors by tapering: [1, 1, 1, 1]


### 2.3.3 final eigen values

In [ ]:
print('Test: Hami_reduced_by_qml\n')
eigvals_reduced_by_qml = np.linalg.eigvals(Hami_reduced_by_qml.matrix)
print('min eigenvalue: ', min(eigvals_reduced_by_qml))

Test: Hami_reduced_by_qml



In [ ]:
print('Test: Hami_reduced_by_tapering\n')
eigvals_reduced_by_tapering = np.linalg.eigvals(Hami_reduced_by_tapering.matrix)
print('min eigenvalue: ', min(eigvals_reduced_by_tapering))